In [ ]:
!pip -q install "griptape[all]" rich schema pandas

import os, re, json
from getpass import getpass

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
except Exception:
    pass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")

In [ ]:
tool_code = r'''
import re, json
from schema import Schema, Literal, Optional
from griptape.tools import BaseTool
from griptape.utils.decorators import activity
from griptape.artifacts import TextArtifact, ErrorArtifact

def _redact(text: str) -> str:
    text = re.sub(r"[\\w\\.-]+@[\\w\\.-]+\\.\\w+", "[REDACTED_EMAIL]", text)
    text = re.sub(r"\\+?\\d[\\d\\-\\s\\(\\)]{7,}\\d", "[REDACTED_PHONE]", text)
    text = re.sub(r"\\b(\\d{4}[\\s-]?){3}\\d{4}\\b", "[REDACTED_CARD]", text)
    return text

class TicketOpsTool(BaseTool):
    @activity(config={"description": "Redact PII", "schema": Schema({Literal("text"): str})})
    def redact_pii(self, params: dict):
        try:
            return TextArtifact(_redact(params["values"]["text"]))
        except Exception as e:
            return ErrorArtifact(str(e))

    @activity(config={"description": "Categorize ticket", "schema": Schema({Literal("text"): str})})
    def categorize(self, params: dict):
        try:
            t = params["values"]["text"].lower()
            if any(k in t for k in ["charged", "refund", "invoice", "billing", "payment"]):
                cat = "billing"
            elif any(k in t for k in ["crash", "error", "bug", "export", "0x"]):
                cat = "bug"
            elif any(k in t for k in ["locked", "password", "login attempts", "unauthorized", "security"]):
                cat = "security"
            elif any(k in t for k in ["account", "profile", "access"]):
                cat = "account"
            else:
                cat = "other"
            return TextArtifact(cat)
        except Exception as e:
            return ErrorArtifact(str(e))

    @activity(config={"description": "Priority and SLA", "schema": Schema({Literal("category"): str, Literal("text"): str, Optional(Literal("channel"), default="web"): str})})
    def priority_and_sla(self, params: dict):
        try:
            cat = params["values"]["category"].lower()
            t = params["values"]["text"].lower()
            channel = params["values"].get("channel", "web")
            if cat == "security" or "urgent" in t or "asap" in t:
                p, sla = 1, "15 minutes"
            elif cat in ["billing", "account"]:
                p, sla = 2, "2 hours"
            elif cat == "bug":
                p, sla = 3, "1 business day"
            else:
                p, sla = 4, "3 business days"
            if channel == "chat" and p > 1:
                p = max(2, p - 1)
            return TextArtifact(json.dumps({"priority": p, "sla_target": sla}))
        except Exception as e:
            return ErrorArtifact(str(e))

    @activity(config={"description": "Escalation payload", "schema": Schema({Literal("ticket_id"): str, Literal("customer"): str, Literal("category"): str, Literal("priority"): int, Literal("sanitized_text"): str})})
    def build_escalation_json(self, params: dict):
        try:
            v = params["values"]
            payload = {
                "summary": f"[{v['category'].upper()}][P{v['priority']}] Ticket {v['ticket_id']} - {v['customer']}",
                "labels": [v["category"], f"p{v['priority']}"],
                "description": v["sanitized_text"],
                "customer": v["customer"],
                "source_ticket": v["ticket_id"]
            }
            return TextArtifact(json.dumps(payload, indent=2))
        except Exception as e:
            return ErrorArtifact(str(e))
'''
with open("/content/ticket_tools.py", "w", encoding="utf-8") as f:
    f.write(tool_code)

import importlib, sys
sys.path.append("/content")
ticket_tools = importlib.import_module("ticket_tools")
TicketOpsTool = ticket_tools.TicketOpsTool
tool = TicketOpsTool()

In [ ]:
TICKETS = [
    {"ticket_id": "TCK-1001", "customer": "Leila", "text": "I was charged twice on my card ending 4432. Please refund ASAP. email: leila@test.com", "channel": "email", "created_at": "2026-02-01T10:14:00Z"},
    {"ticket_id": "TCK-1002", "customer": "Rohan", "text": "App crashes every time I try to export. Screenshot shows error code 0x7f. My phone: +1 514-555-0188", "channel": "chat", "created_at": "2026-02-01T10:20:00Z"},
    {"ticket_id": "TCK-1003", "customer": "Mina", "text": "Need invoice for January. Also update billing address to 21 King St, Montreal.", "channel": "email", "created_at": "2026-02-01T10:33:00Z"},
    {"ticket_id": "TCK-1004", "customer": "Sam", "text": "My account got locked after password reset. I’m seeing login attempts I don't recognize. Please help urgently.", "channel": "web", "created_at": "2026-02-01T10:45:00Z"}
]

In [4]:
from griptape.structures import Agent
from griptape.drivers.prompt.openai import OpenAiChatPromptDriver

prompt_driver = OpenAiChatPromptDriver(model="gpt-4.1")
agent = Agent(prompt_driver=prompt_driver, tools=[tool])

def run_ticket(ticket: dict) -> dict:
    sanitized = tool.redact_pii({"values": {"text": ticket["text"]}}).to_text()
    category = tool.categorize({"values": {"text": sanitized}}).to_text().strip()
    pr_sla = json.loads(tool.priority_and_sla({"values": {"category": category, "text": sanitized, "channel": ticket["channel"]}}).to_text())
    escalation = tool.build_escalation_json({"values": {"ticket_id": ticket["ticket_id"], "customer": ticket["customer"], "category": category, "priority": int(pr_sla["priority"]), "sanitized_text": sanitized}}).to_text()
    prompt = f"""
You are a senior support lead. Produce:
1) A customer-facing reply
2) Internal notes
3) Escalation decision

Ticket:
- id: {ticket['ticket_id']}
- customer: {ticket['customer']}
- channel: {ticket['channel']}
- category: {category}
- priority: {pr_sla['priority']}
- SLA target: {pr_sla['sla_target']}
- sanitized_text: {sanitized}

Output in Markdown.
"""
    out = agent.run(prompt).to_text()
    return {"ticket_id": ticket["ticket_id"], "category": category, "priority": pr_sla["priority"], "sla_target": pr_sla["sla_target"], "escalation_payload_json": escalation, "agent_output_markdown": out}

In [5]:
results = [run_ticket(t) for t in TICKETS]

for r in results:
    print("\n" + "=" * 88)
    print(f"{r['ticket_id']} | category={r['category']} | P{r['priority']} | SLA={r['sla_target']}")
    print(r["escalation_payload_json"])
    print(r["agent_output_markdown"])

Tool loaded OK: <class 'ticket_tools.TicketOpsTool'>
